In [18]:
import requests
from bs4 import BeautifulSoup
import urllib3
import pandas as pd
import time
from requests.exceptions import RequestException
from collections import Counter
import os

urllib3.disable_warnings()  # Warning message 제거

# 검색어 설정
search_query = 'resnet'

# 연도 범위 설정
years = range(2015, 2025)

# 연도별 크롤링 및 CSV 파일 생성
for search_year in years:
    data = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
        'Referer': 'https://www.riss.kr'
    }

    # 페이지 순회
    for i in range(0, 1000):  # 페이지 계산
        param = {
            'isDetailSearch': 'N',
            'searchGubun': 'true',
            'viewYn': 'OP',
            'strQuery': search_query,
            'exQuery': f'regnm:KCI등재◈regnm:KCI우수등재◈regnm:KCI등재후보◈pyear:{search_year}◈',
            'exQueryText': f'등재정보 [KCI등재]@@regnm:KCI등재◈등재정보 [KCI우수등재]@@regnm:KCI우수등재◈등재정보 [KCI등재후보]@@regnm:KCI등재후보◈발행연도 [{search_year}]@@pyear:{search_year}◈',
            'order': '/DESC',
            'onHanja': 'false',
            'strSort': 'RANK',
            'iStartCount': i * 100,  # 페이지 계산 (iStartCount는 0, 100, 200, ... 형식으로 증가)
            'fsearchMethod': 'search',
            'sflag': 1,
            'isFDetailSearch': 'N',
            'icate': 're_a_kor',
            'colName': 're_a_kor',
            'pageScale': 100,
            'isTab': 'Y',
            'query': search_query
        }

        try:
            # 메인 검색 결과 페이지 요청
            response = requests.get("https://www.riss.kr/search/Search.do", params=param, headers=headers, timeout=10, verify=False)
            response.raise_for_status()  # HTTP 오류 발생 시 예외
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            articles = soup.select(".srchResultListW>ul>li")  # 검색 결과 항목들 선택

            if not articles:  # 더 이상 결과가 없으면 종료
                print(f"{search_year}년 데이터 크롤링 완료.")
                break

            for article in articles:
                try:
                    title = article.select_one(".title > a").text.strip()
                    link = 'https://www.riss.kr/' + article.select_one(".title > a").attrs['href']

                    # 상세 페이지 요청
                    response_detail = requests.get(link, headers=headers, timeout=10, verify=False)
                    response_detail.raise_for_status()
                    html_detail = response_detail.text
                    soup_detail = BeautifulSoup(html_detail, 'html.parser')
                    year = soup_detail.find('span', string='발행연도').find_next_sibling().text.strip()

                    # 주제어 수집
                    if soup_detail.find('span', string='주제어'):
                        keywords = soup_detail.find('span', string='주제어').find_next_sibling().text.split(';')
                        keywords = [keyword.strip() for keyword in keywords]
                    else:
                        keywords = None

                    # 초록 내용 수집
                    abstract = None  # 국문 초록 초기화
                    multilingual_abstract = None  # 다국어 초록 초기화

                    abstract_sections = soup_detail.select(".content > div")
                    for section in abstract_sections:
                        title_tag = section.select_one("p.title")
                        if title_tag:
                            title_text = title_tag.text.strip()
                            # 국문 초록
                            if "국문 초록" in title_text:
                                text_off = section.select_one(".text.off > p")
                                if text_off:
                                    abstract = text_off.text.strip()
                            # 다국어 초록
                            elif "다국어 초록" in title_text or "Multilingual" in title_text:
                                text_off = section.select_one(".text.off > p")
                                if text_off:
                                    multilingual_abstract = text_off.text.strip()

                    print(title, year, keywords, abstract, multilingual_abstract)
                    data.append([title, year, keywords, abstract, multilingual_abstract])
                except Exception as e:
                    print(f"세부 데이터 수집 중 오류 발생: {e}")

            # 페이지 간 지연 시간 추가
            time.sleep(2)  # 2초 대기

        except RequestException as e:
            print(f"페이지 요청 중 오류 발생: {e}. 5초 후 재시도합니다.")
            time.sleep(5)  # 재시도 전 대기

    # 결과 데이터프레임으로 변환
    df = pd.DataFrame(data, columns=['title', 'date', 'keywords', 'abstract', 'multilingual_abstract'])

    # 주제어가 None인 경우 빈 리스트로 변환
    df['keywords'] = df['keywords'].apply(lambda x: x if x is not None else [])

    # 주제어 빈도수 계산
    all_keywords = [keyword for row in df['keywords'] for keyword in row]  # 모든 주제어를 하나의 리스트로 결합
    keyword_counts = Counter(all_keywords)  # 빈도수 계산

    # 주제어 빈도수 데이터프레임 생성
    keyword_df = pd.DataFrame(keyword_counts.items(), columns=['keyword', 'count']).sort_values(by='count', ascending=False)

    # 결과 출력 설정
    pd.set_option('display.max_columns', None)  # 열을 모두 출력
    pd.set_option('display.max_rows', None)  # 행을 모두 출력
    pd.set_option('display.max_colwidth', None)  # 긴 데이터도 모두 출력

    # 결과 출력
    print("=== 데이터프레임 ===")
    print(df)
    print("\n=== 주제어 빈도수 ===")
    print(keyword_df)

    # 검색어를 파일 이름에 안전하게 사용할 수 있도록 변환
    safe_search_query = search_query.replace(' ', '_')

    # 파일명 생성
    csv_filename = f"{safe_search_query}_{search_year}_academic_riss.csv"
    keyword_csv_filename = f"k_{safe_search_query}_{search_year}_academic_riss.csv"

    # 폴더 경로 설정
    academic_riss_folder = f"{search_year}_academic_riss"
    keyword_riss_folder = "k_academic_riss"

    os.makedirs(academic_riss_folder, exist_ok=True)
    os.makedirs(keyword_riss_folder, exist_ok=True)

    # 파일 저장
    df.to_csv(f"{academic_riss_folder}/{csv_filename}", index=False)
    keyword_df.to_csv(f"{keyword_riss_folder}/{keyword_csv_filename}", index=False)

    print(f"데이터가 저장되었습니다: {csv_filename}")
    print(f"데이터가 저장되었습니다: {keyword_csv_filename}")

print("모든 데이터 크롤링 및 저장 완료.")


딥러닝을 이용한 고압 수소에 의하여 손상된 Acrylonitrile Butadiene Rubber (NBR) 단면 균열 탐지 2021 ['딥러닝(Deep learning)', '균열 탐지(Crack Detection)', '인공신경망(Artificial neural network)', '객체인식(Object detection)', '반 자동화 라벨링(Semi-automated labeling)'] None Demand for understanding material behaviors in hydrogen rich environment rises due to the extended use of hydrogen gas. Organic material such as rubber used in O-rings for hydrogen containers displays pore-like cracks when it is exposed extensively to high pressure hydrogen. To find the optimal organic material in a certain hydrogen rich environment, a method for localizing and analyzing pore-like cracks must be devised. This paper deals with a new method to detect high pressure hydrogen induced cracksusing deep learning algorithms. In this study, acrylonitrile Butadiene Rubber (NBR) was exposed to hydrogen of 96.6MPa for 24 hours. Images were taken of the appearing pore like cracks magnified by 100. For bounding box labeling a semi-automated labeling was used 